In [61]:
import os
import pandas as pd
import numpy as np
import env
from skimpy import skim


In [62]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    function to generate a url for querying the codeup database
    accepts a database name (string) and requires an env.py file with 
    username, host, and password.

    Returns an url as a string  
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [63]:
def get_superstore():
    """
    Retrieve locally cached data .csv file for the superstore dataset
    If no locally cached file is present retrieve the data from the codeup database server
    Keyword arguments: none
    Returns: DataFrame

    """
    filename = "superstore.csv"

    # if file is available locally, read it
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    
    else:
    # if file not available locally, acquire data from SQL database
    # and write it as csv locally for future use 
        df = pd.read_sql('''
                            select * from orders
                            join customers USING (`Customer ID`)
                            join categories USING (`Category ID`)
                            join products USING (`Product ID`)
                            join regions USING (`Region ID`)
                         ''', get_connection('superstore_db'))

    # Write that dataframe to disk for later. This cached file will prevent repeated large queries to the database server.
    df.to_csv(filename, index=False)
    return df

In [64]:
def prepare_superstore(df):
    df.columns = df.columns.str.replace(' ', '_')                # add underscores to column names
    df.columns = df.columns.str.lower()                          # lowercase columns
    df.order_date = pd.to_datetime(df.order_date)                # convert order_date to datetime object
    df.ship_date = pd.to_datetime(df.ship_date)                  # convert ship_date to datetime object
    df['process_time'] = (df.ship_date - df.order_date).dt.days  # create process time, days between order and ship
    df['profit_margin'] = df.profit / df.sales                   # profit margin as a decimal.  can be negative
    df = df.set_index('order_date')                              # set order_date as index
    df['quarter'] = df.index.quarter                             # create column for quarter
    df['year'] = df.index.year                                   # create column for year
    return df

In [65]:
df = get_superstore()

In [66]:
df = prepare_superstore(df)

In [67]:
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1734   │ │ string      │ 13    │                                                          │
│ │ Number of columns │ 25     │ │ float64     │ 6     │                                                          │
│ └───────────────────┴────────┘ │ int64       │ 5     │                                                          │
│                                │ datetime64  │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %   ┃ mean     ┃ sd      ┃ p0      ┃ p25     ┃ p75     ┃ p100    ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩  │
│ │ region_id          │    0 │      0 │      2.7 │     1.1 │       1 │       2 │       4 │       4 │  ▄ █ ▆█  │  │
│ │ category_id        │    0 │      0 │      8.1 │     3.5 │       1 │       6 │      11 │      17 │  ▃▆▄█▃▁  │  │
│ │ postal_code        │    0 │      0 │    54000 │   32000 │    1800 │   19000 │   86000 │   99000 │  ▅▅▄▂▄█  │  │
│ │ sales              │    0 │      0 │      230 │     530 │     1.1 │      17 │     210 │    8200 │    █     │  │
│ │ quantity           │    0 │      0 │      3.8 │     2.3 │       1 │       2 │       5 │      14 │   █▃▂▁   │  │
│ │ discount           │    0 │      0 │     0.16 │    0.21 │       0 │       0 │     0.2 │     0.8 │  █▆ ▁ ▁  │  │
│ │ profit             │    0 │      0 │       22 │     200 │   -3800 │     1.7 │      27 │    3200 │      █   │  │
│ │ process_time       │    0 │      0 │      3.9 │     1.7 │       0 │       3 │       5 │       7 │  ▃▅▃█▆▅  │  │
│ │ profit_margin      │    0 │      0 │     0.12 │    0.45 │    -2.8 │   0.064 │    0.36 │     0.5 │     ▁▁█  │  │
│ │ quarter            │    0 │      0 │      2.9 │     1.1 │       1 │       2 │       4 │       4 │  ▃ ▅ ▆█  │  │
│ │ year               │    0 │      0 │     2000 │     1.1 │    2000 │    2000 │    2000 │    2000 │  ▅ ▆ ██  │  │
│ └────────────────────┴──────┴────────┴──────────┴─────────┴─────────┴─────────┴─────────┴─────────┴──────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name            ┃ NA     ┃ NA %      ┃ first               ┃ last                ┃ frequency        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ ship_date              │      0 │         0 │     2014-01-08      │     2018-01-05      │ None             │  │
│ └────────────────────────┴────────┴───────────┴─────────────────────┴─────────────────────┴──────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                  ┃ NA     ┃ NA %       ┃